# Vehicle Detection and Tracking

---

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import os
import time

%matplotlib inline

## Load Data

In [4]:
test_images = [mpimg.imread('test_images/' + file) for file in os.listdir('test_images')]

In [5]:
# These are all 64x64 png images

vehicles_folders = ['data/vehicles/GTI_Far/', 'data/vehicles/GTI_Left/', 'data/vehicles/GTI_MiddleClose/', 'data/vehicles/GTI_Right/', 'data/vehicles/KITTI_extracted/']
nonvehicles_folders = ['data/non-vehicles/Extras/', 'data/non-vehicles/GTI/']


vehicles = [mpimg.imread(folder + file) for folder in vehicles_folders for file in os.listdir(folder)]
non_vehicles = [mpimg.imread(folder + file) for folder in nonvehicles_folders for file in os.listdir(folder)]

features_raw = np.concatenate((vehicles, non_vehicles))

## Train SVM

In [13]:
def feature_pipeline(image):
    cv2.resize(image, (16, 16))
    yuv = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    ch0 = yuv[:,:,0]
    ch1 = yuv[:,:,1]
    ch2 = yuv[:,:,2]
    ch0_features = hog(ch0, cells_per_block=(2,2), pixels_per_cell=(4,4))
    ch1_features = hog(ch1, cells_per_block=(2,2), pixels_per_cell=(4,4))
    ch2_features = hog(ch2, cells_per_block=(2,2), pixels_per_cell=(4,4))
    hog_features = np.concatenate((ch0_features, ch1_features, ch2_features))
    return hog_features

In [ ]:
start = time.time()

features = [feature_pipeline(image) for image in features_raw]
labels = np.concatenate((np.ones(len(vehicles)), np.zeros(len(non_vehicles))))

scaler = StandardScaler()
scaler.fit(features)
features_scaled = scaler.transform(features)

end = time.time()

print('Runtime (s):', end - start)

Runtime (s): 250.1805055141449


In [ ]:
start = time.time()

split_index = int(len(features_scaled) * .8)
X_train = features_scaled[0: split_index]
X_test = features_scaled[split_index:]
y_train = labels[0:split_index]
y_test = labels[split_index:]

svc = LinearSVC()
svc.fit(X_train, y_train)
accuracy = svc.score(X_test, y_test)

end = time.time()

print('Test Accuracy: ', accuracy)
print('Model runtime (s):', end - start)

## Pipeline

In [1]:
def pipeline(image):
    region_of_interest = image[360:]
    windows = [region_of_interest[y1:y2,x1:x2] for x1,y1,x2,y2 in sliding_windows(region_of_interest.shape)]
    windows = np.concatenate((windows,))
    print(windows.shape)
    return image

def sliding_windows(shape):
    return [((x,y, x+64, y+64)) for y in range(0, shape[0], 32) for x in range(0, shape[1], 32)]

In [2]:
for image in test_images:
    plt.figure(figsize=(15,5))
    plt.subplot(121)
    plt.imshow(image)
    plt.subplot(122)
    plt.imshow(pipeline(image), cmap='gray')
    break

NameError: name 'test_images' is not defined